In [659]:
import torch.optim as optim
import torchmetrics
import torch
import torch.nn.functional as F
import igl 
import networkx as nx 
import numpy as np # np.linalg.eig
import scipy as sp
from meshplot import plot, subplot, interact
import meshplot as mp
import time

# igl 
import os
root_folder = os.getcwd()

**Hyper Parameters**

In [644]:
mesh_dir = '\SHREC11'
NUM_MESHES = 600 
NUM_GRPS = 30
path_to_labels = os.path.join(root_folder, "SHREC11", "labels.txt")
train_size = int(.8*NUM_MESHES) # set aside train (80% )and test (20%) data 

# set values of t 
# ts= [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1] # TODO: add values larger than 1 

# tODO: have more values of T 
ts=np.exp(np.linspace(-3,3,20))

In [645]:
# convert mesh to networkx graph

def do_task(num):
    v, f = igl.read_triangle_mesh(os.path.join(root_folder, "SHREC11", "T"+str(num)+".obj"))
    # Mesh in (v,f)
    adj_mat = igl.adjacency_matrix(f) # this gets me the adj_mat for this mesh 
    # print("type", type(adj_mat)) # type <class 'scipy.sparse._csc.csc_matrix'>
    
    G = nx.from_scipy_sparse_array(adj_mat) # creates a new graph from an adj matrix given as a Scipy sparse array 
    # print("G", G)
    
    L = nx.laplacian_matrix(G).toarray() # get Laplacian matrix  
    # print("L", L)
    
    # get eigenvalues from graph Laplacian 
    eigen_values = np.linalg.eigvals(L)
    # print("eigen_values", eigen_values)
    # print("eigen_values type", type(eigen_values)) # ndarray
    # print("eigen_values len", len(eigen_values)) # 252 nodes hence len is 252 
    
    # compute e^t*eigen_value
    
    HKS = [] 
    
    for t in ts:
        t_eigen_values = t*eigen_values # an array where each lambda is multiplied by t 
        # print("t_eigen_values", t*eigen_values)

        h_t = np.mean(np.exp(-1*t_eigen_values)) # get average to compute h(t) 
        # print("h_t", h_t)
        
        HKS.append(h_t) 
        
    return HKS 
    


In [646]:
HKS_all_do_task = [] 

start_time = time.time()
for i in range(600):
    
    HKS = do_task(i)
    HKS_all_do_task.append(HKS)
    
end_time = time.time()

time_taken = end_time-start_time

print('Execution time:', time_taken, 'seconds')
    
print("HKS len", len(HKS_all_do_task))


Execution time: 22.11951994895935 seconds
HKS len 600


# obtain eigenvalues for one mesh
# input: mesh number referencing a mesh
# output: np array of the eigenvalues of that particular mesh
def calc_eigvals(num):
    
    # convert mesh to networkx graph
    v, f = igl.read_triangle_mesh(os.path.join(root_folder, "SHREC11", "T"+str(num)+".obj"))
    # Mesh in (v,f)
    adj_mat = igl.adjacency_matrix(f) # this gets me the adj_mat for this mesh 
    # print("type", type(adj_mat)) # type <class 'scipy.sparse._csc.csc_matrix'>
    
    G = nx.from_scipy_sparse_array(adj_mat) # creates a new graph from an adj matrix given as a Scipy sparse array 
    # print("G", G)
    
    L = nx.laplacian_matrix(G).toarray() # get Laplacian matrix  
    # print("L", L)
    
    # get eigenvalues from graph Laplacian 
    eigen_values = np.linalg.eigvals(L)
    # print("eigen_values: ", eigen_values)
    # print("eigen_values type: ", type(eigen_values)) # ndarray
    # print("eigen_values len: ", len(eigen_values)) # 252 nodes hence len is 252 
    
    return eigen_values

# each index of list corresponds to mesh number 
eig_vals_all_lst = []

start_time = time.time()
for i in range(600):
    # obtain eigen values for a mesh
    eigen_values = calc_eigvals(i)
    # append mesh's eigen values to big list 
    eig_vals_all_lst.append(eigen_values)
    
end_time = time.time()

time_taken = end_time-start_time

print('Execution time to obtain eigen values:', time_taken, 'seconds')

# print("len of big list",len(eig_vals_all_lst))
# print("type of big list",type(eig_vals_all_lst))
# print("type of one of the elements",type(eig_vals_all_lst[0]))
# print(eig_vals_all_lst[0])

# calculates HKS for one mesh
# input: mesh number referencing a mesh
# output: HKS for that particular mesh 
def calc_HKS(num):
   
    # compute e^t*eigen_value
    
    HKS = [] 
    eig_values = eig_vals_all_lst[num]
    
    
    for t in ts:
        t_eigen_values = t*eig_values
        # print("t_eigen_values", t*eigen_values)

        h_t = np.mean(np.exp(-1*eig_values)) # get average to compute h(t) 
        
        # print("h_t", h_t)
        
        HKS.append(h_t) 
        
    return HKS 

HKS_all = [] 

start_time = time.time()
for i in range(600):
    
    HKS = calc_HKS(i)
    HKS_all.append(HKS)
    
end_time = time.time()

time_taken = end_time-start_time

print('Execution time:', time_taken, 'seconds')
    
print("HKS len", len(HKS_all))


In [656]:
print("do task", type(HKS_all_do_task))
print("non do task", type(HKS_all))
# print("do task", HKS_all_do_task[:10])

a = np.array(HKS_all_do_task)
b = np.array(HKS_all)

print("do task", type(a))
print("non do task", type(b))

print(a-b)


do task <class 'list'>
non do task <class 'list'>
do task <class 'numpy.ndarray'>
non do task <class 'numpy.ndarray'>
[[ 0.69503992  0.62205799  0.53578197 ... -0.04242218 -0.04397942
  -0.04530437]
 [ 0.69139735  0.61881573  0.53312126 ... -0.04786033 -0.05054707
  -0.05254971]
 [ 0.69398549  0.62117232  0.53514499 ... -0.04646591 -0.04833545
  -0.04985514]
 ...
 [ 0.69357913  0.62088386  0.5350269  ... -0.04722566 -0.04901835
  -0.05046278]
 [ 0.69240441  0.61981156  0.53410484 ... -0.04641691 -0.04830271
  -0.04997075]
 [ 0.6910785   0.61847254  0.53274427 ... -0.04751314 -0.05032086
  -0.05247441]]


In [616]:
#I have a list that contains the HKS vectors for each mesh, and I will 
# convert that into a tensor. The objects numbered 0 through 599 are scrambled
# and not organised by category but there is a labels.txt file that groups them
# by category, so using that labels.txt file, I will create a list parallel to the
# HKS vectors that contains the group number for each mesh, and convert that into a tensor. 


In [632]:
# organise labels - taken from Davidson and Richard's code 
def readLbl(size,fileName):
    #takes in file name, returns the labels as an array
    file1 = open(fileName, 'r')
    Lines = file1.readlines()

    count = 0

    lbls = np.empty([size])
    obj_order_by_grp = np.empty([size]) # list of obj name ordered by grp no.
    lbls_order_by_grp = np.empty([size])  # list of grps ordered by grp no.
    # Strips the newline character
    for line in Lines:
        count += 1
        text = line.strip()[1:].split('.')
        text[1] = text[1].split(' ')[1]
        
        # list of obj name ordered by grp no.
        obj_order_by_grp[count-1] = int(text[0])
        
        # list of grps ordered by grp no.
        lbls_order_by_grp[count-1] = int(text[1])
        
        # parallel list to HKS - ordered by object no.
        lbls[int(text[0])] = int(text[1])
        #print("Line{}: {}".format(count, )))
        
        # file1.close()
    return lbls, obj_order_by_grp, lbls_order_by_grp

In [633]:
# fName_labels = mesh_dir + 'labels.txt'
labels_np, obj_order_by_grp, lbls_order_by_grp = readLbl(NUM_MESHES, path_to_labels)

# print("obj_order_by_grp", obj_order_by_grp)
# print("lbls_order_by_grp", lbls_order_by_grp)

**BINARY**

In [634]:
# obtain labels of group 0 and 1 from larger np.arr, put them in labels_binary
# => get a np.arr that is [0,0..0,1,1..1]
labels_binary = lbls_order_by_grp[:40] # type: ndarray 

print("labels_binary", type(labels_binary))
print("labels_binary", labels_binary)

labels_binary <class 'numpy.ndarray'>
labels_binary [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


**Binary - bigger datatset**

In [777]:
labels_all=lbls_order_by_grp # type: ndarray 

# print("labels_multi", type(labels_multi))
# print("labels_multi", labels_multi)

**BINARY**

In [635]:
# get a np.arr of meshes whose groups are 0 and 1
# this is parallel to [0,0..0,1,1..1]
obj_order_by_grp_binary = obj_order_by_grp[:40]

# convert all floats in np array to int 
a = obj_order_by_grp_binary.astype(int)

# obtain HKS belonging to group 0 and 1
HKS_binary_ls = [] 
for obj in a.tolist(): # tolist() converts type from np.arr to python list 
    el = HKS_all_do_task[obj]
    HKS_binary_ls.append(el)

# convert HKS_binary_ls back to np.array 
HKS_binary = np.array(HKS_binary_ls)

**Binary - bigger datatset**

In [778]:
obj_order_by_grp_all = obj_order_by_grp

# convert all floats in np array to int 
a = obj_order_by_grp_all.astype(int)

HKS_all_cat_ls = [] 
for obj in a.tolist(): # tolist() converts type from np.arr to python list 
    el = HKS_all_do_task[obj]
    HKS_all_cat_ls.append(el)

# convert HKS_binary_ls back to np.array 
HKS_all_cat = np.array(HKS_all_cat_ls)

**BINARY**

In [ ]:
# print("pre shuffle:", labels_binary)
# print("pre shuffle:", HKS_binary)

# shuffle two np arrays together
rand_indexes = np.arange(len(labels_binary))
np.random.shuffle(rand_indexes)
labels_binary=labels_binary[rand_indexes]
HKS_binary=HKS_binary[rand_indexes]

# print("post shuffle:", labels_binary)
# print("post shuffle:", HKS_binary)

**Binary - bigger datatset**

In [779]:
grps = []
HKS_grps = [] # parallel to grps 

# separates big list into 30 smaller lists 
for i in range(30):
    grps.append(labels_all[i*20:i*20+20])
    HKS_grps.append(HKS_all_cat[i*20:i*20+20])

In [786]:
# print("pre-shuffle",HKS_grps[13][:3])

In [795]:
labels_all_train_ver2 = []
labels_all_test_ver2 = []

HKS_all_train_ver2 = []
HKS_all_test_ver2 = []

# shuffle two np arrays together
# then split into train and test set 
for i in range(30):
    # shuffle
    rand_indexes = np.arange(20)
    np.random.shuffle(rand_indexes)
    grps[i]=grps[i][rand_indexes]
    HKS_grps[i]=HKS_grps[i][rand_indexes]
    
    # split into train and test set 
    labels_train_ver2 = torch.tensor(grps[i][:int(.8*20)]).float()
    # torch.reshape(labels_train_ver2, (32,1))
    labels_all_train_ver2.append(labels_train_ver2)
    
    labels_test_ver2 = torch.tensor(grps[i][int(.8*20):]).float()
    labels_all_test_ver2.append(labels_test_ver2)
    
    HKS_train_ver2 = torch.tensor(HKS_grps[i][:int(.8*20)]).float()
    HKS_all_train_ver2.append(HKS_train_ver2)
    
    HKS_test_ver2 = torch.tensor(HKS_grps[i][int(.8*20):]).float()
    HKS_all_test_ver2.append(HKS_test_ver2)

In [796]:
# print("pre-shuffle",HKS_grps[13][:3])

In [797]:
#labels_all_train_ver2_unsq = labels_all_train_ver2.unsqueeze(1)

In [805]:
# 30 choose 2 -> get every single combination of 2 categories 
# and append to list
size_grps=np.arange(30)
lst = []
count = 0
for i in range(30):
    for j in range(i+1, 30):
        pointer1 = i
        pointer2 = j
        x = size_grps[pointer1] # a shld point to array 
        y = size_grps[pointer2]
        count += 1
        lst.append((x,y))
print("count: ", count)

mytuple=lst[0]
print("type", type(mytuple[0])) # tuple
print("type", type(mytuple[1])) # tuple
print("type", mytuple[0]) # tuple
print("type", mytuple[1]) # tuple

# append actual np arrays together

labels_v2 = []

# goal: combine pairs into one tensor
# append tensors to list 

# obtain tensor based on tuple
# combine tensors 


tensor_list = []
for mytuple in lst:
    grpA=mytuple[0]
    grpB=mytuple[1]
    
    tensor_pair = torch.cat((labels_all_train_ver2[grpA],labels_all_train_ver2[grpB]))
    tensor_list.append(tensor_pair)
    
    torch.where(grpA)
    
    


count:  435
type <class 'numpy.int32'>
type <class 'numpy.int32'>
type 0
type 1


## **BINARY CLASSIFIER**

In [637]:
# training size is hardcoded bc small dataset 
labels_binary_train = torch.tensor(labels_binary[:int(.8*40)]).float()
torch.reshape(labels_binary_train, (32,1))
labels_binary_test = torch.tensor(labels_binary[int(.8*40):]).float()

HKS_binary_train = torch.tensor(HKS_binary[:int(.8*40)]).float()
HKS_binary_test = torch.tensor(HKS_binary[int(.8*40):]).float()

In [638]:
labels_binary_train_unsqueezed = labels_binary_train.unsqueeze(1)

In [639]:
#TODO: replace Sigmoid activations with SiLU, ELU (those can perform better than Sigmoid) 
# ReLU are also commonly used, they have same problem with Sigmoid 

# make NN deeper (more layers) and wider (higher dimensions) (layer size) 

# binary classifier 
hks_binary_classifier = torch.nn.Sequential( 
    torch.nn.BatchNorm1d(len(ts)), #  helps NN converge faster 
    torch.nn.Linear(len(ts), 120), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 120), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 120),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 120),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 120),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 120), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(),
    torch.nn.Linear(120, 64), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(),
    torch.nn.Linear(64, 1), #(x,y) -> y is the size of my output I'm doing a linear classifier (binary now, later 25 categories)
    # torch.nn.Linear()
)

In [640]:
criterion=torch.nn.BCEWithLogitsLoss()

In [641]:
# later: 
# how to train in batches for PyTorch for 600 (look this up)
# in pytorch u create a dataset obj and data loader obj that loads from that dataset
# purpose: give u pieces of my dataset at a time 

In [642]:
# training loop 
optimizer = optim.Adam(hks_binary_classifier.parameters(), lr = 0.0001)
for i in range(2000):
    optimizer.zero_grad()
    # print(HKS_binary_train)
    output = hks_binary_classifier.forward(HKS_binary_train)
    # print(output) 
    loss = criterion(output,labels_binary_train_unsqueezed)
    loss.backward()
    if i%100 == 0:
        print(loss) # print loss every 100 (so not printing every single round) (i mod 30 0) 
    optimizer.step()

tensor(0.6949, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.5020, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0019, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0006, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0003, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0002, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(0.0001, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(9.2380e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(6.9080e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(5.3461e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(4.2515e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(3.4534e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(2.8537e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(2.3911e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(2.0290e-05, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
tensor(1

**Binary w/ bigger dataset**

In [ ]:
# training loop 
optimizer = optim.Adam(hks_binary_classifier.parameters(), lr = 0.0001)
for i in range(2000):
    optimizer.zero_grad()
    # print(HKS_binary_train)
    output = hks_binary_classifier.forward(HKS_binary_train)
    # print(output) 
    loss = criterion(output,labels_binary_train_unsqueezed)
    loss.backward()
    if i%100 == 0:
        print(loss) # print loss every 100 (so not printing every single round) (i mod 30 0) 
    optimizer.step()

In [560]:
# u do the test only once 
# it outputs predicted values
# actual-predicted for each value
# get the avg of all the values 
# hks_binary_classifier.forward(HKS_binary_test)

In [673]:
preds = hks_binary_classifier.forward(HKS_binary_test)
adj_preds = torch.where(preds > 0, 1., 0.)

In [674]:
labels_binary_test_unsqueezed = labels_binary_test.unsqueeze(1)
target = labels_binary_test_unsqueezed

In [676]:
metric = torchmetrics.classification.BinaryAccuracy()
# prediction, target 
print("adj_preds", adj_preds)
print("target", target)
metric(adj_preds, target)

accuracy = adj_preds-target

print("accuracy", accuracy)


adj_preds tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]])
target tensor([[1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]])
accuracy tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


## **MULTI CLASS CLASSIFIER**

In [731]:
# cast to tensor for all 600 groups
# note: everything is scrambled (group wise)
# arranged in order from T0-T599 
labels_train = torch.tensor(labels_np[:train_size]).float()
labels_test = torch.tensor(labels_np[train_size:]).float()

HKS_all_train = torch.tensor(HKS_all_do_task[:train_size]).float()
HKS_all_test = torch.tensor(HKS_all_do_task[train_size:]).float()

In [732]:
# cast to int otherwise error message: (one_hot is only applicable to index tensor.)
labels_train_one_hot = torch.nn.functional.one_hot(labels_train.to(torch.int64))

In [733]:
# labels_train_one_hot_unsq = labels_train_one_hot.unsqueeze(1)

In [734]:
# multi-class classifier 
hks_multi_classifier = torch.nn.Sequential( 
    torch.nn.BatchNorm1d(len(ts)), #  helps NN converge faster 
    torch.nn.Linear(len(ts), 150), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(150, 150), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(150, 150),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(150, 150),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(150, 120),
    torch.nn.SiLU(), # activation function, can use sigmoid etc 
    torch.nn.Linear(120, 100), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(),
    torch.nn.Linear(100, 64), # takes in vector of mine is 4, 16 means weight layer 
    torch.nn.SiLU(),
    torch.nn.Linear(64, 30), #(x,y) -> y is no. of categories 
    torch.nn.Softmax()
)

In [735]:
criterion=torch.nn.BCEWithLogitsLoss()

In [738]:
# training loop 
optimizer = optim.Adam(hks_multi_classifier.parameters(), lr = 0.0002)

start_time = time.time()

for i in range(2000):
    optimizer.zero_grad()
    # print(HKS_binary_train)
    output = hks_multi_classifier.forward(HKS_all_train)
    # print(output) 
    loss = criterion(output,labels_train_one_hot.float())
    loss.backward()
    if i%100 == 0:
        print(str(i), loss) # print loss every 100 (so not printing every single round) (i mod 30 0) 
    optimizer.step()

end_time = time.time()

time_taken = end_time-start_time

print('Execution time:', time_taken, 'seconds')
print('Execution time:', time_taken/60, 'minutes')

0 tensor(0.6906, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
100 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
200 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
300 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
400 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
500 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
600 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
700 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
800 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
900 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
1000 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
1100 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
1200 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
1300 tensor(0.6896, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
1400 tensor(0.6896, grad_fn=<BinaryCrossEntrop